In [ ]:
graphics_toolkit("gnuplot"); % use if "plot" does not work

## Finite element method (FEM) in 2d
### Anisotropic material
Solve the following 2d diffusion boundary value problem using linear finite elements. First consider $k_0=0$ (without reactive term).

In [ ]:
%% 2d boundary value problem:
%   -div(K(x)*grad(u(x))) + k0(x)u(x) = f(x) inside of the domain
%   u(x)=U(x) at GammaD
%   K(x)*(du/dn)(x)=g(x) at GammaN

Uhat = 0.5;
k0 = 10;
f = 0;
f_modified = f+k0*Uhat;

Consider rectangular domain $(0,L_1)\times(0,L_2)$ and its regular triangulation, i.e. prepare input matrices NODE, ELEM:

In [ ]:
%% example: preparation of a simple network
%  triangulation of a rectangular domain (0,L1)x(0,L2)
nx=30; ny=30;
[coords1,coords2]=meshgrid(linspace(0,1,nx),linspace(0,1,ny)); % meshgrid
coords1=coords1(:); coords2=coords2(:);
NODE=[coords1 coords2];
n_NODE=size(NODE,1);
ELEM=rectangle_triangulation(nx,ny);
n_ELEM=size(ELEM,1);
AREAS=polyarea(coords1(ELEM),coords2(ELEM),2);

Preparation of boundary conditions and material inputs:

In [ ]:
%% boundary conditions inputs
DBOUNDARY=((coords1==0)|(coords1==1)|(coords2==0)|(coords2==1));
DVALUE=zeros(sum(DBOUNDARY),1); % values of U(x) in boundary nodes
NBOUNDARY=[];    
NVALUE=ones(size(NBOUNDARY,1),1); % values of g(x) on boundary edges!
FREENODE=true(n_NODE,1); FREENODE(DBOUNDARY)=false;

%% other input data
alpha=pi/3; % axis
U=[cos(alpha) sin(-alpha); sin(alpha) cos(-alpha)];
D=diag([100 1]);
K=U*D*U'
MATERIALS=ones(n_ELEM,1)*reshape(U*D*U',1,4); % anisotropic material
MATERIALS0=k0*ones(n_ELEM,1);
F=f_modified*ones(n_ELEM,1);

Construct FEM matrix and right hand side vector using local matrices and vectors:

In [ ]:
%% construction of global matrix and rhs
A=zeros(n_NODE);
b=zeros(n_NODE,1);
for i=1:n_ELEM
    % local "stiffness" matrix
    x=NODE(ELEM(i,:),:);
    B_=[x(2,2)-x(3,2) x(3,2)-x(1,2) x(1,2)-x(2,2);
        x(3,1)-x(2,1) x(1,1)-x(3,1) x(2,1)-x(1,1)];
    #A_local=(B_')*reshape(MATERIALS(i,:),2,2)*B_/4/AREAS(i);
    A_local=(B_')*K*B_/4/AREAS(i); % only if K in constant (homogeneous material)
    % local "mass" matrix
    J=abs(2*AREAS(i));
%     DF=[x(2,1)-x(1,1) x(3,1)-x(1,1); 
%         x(2,2)-x(1,2) x(3,2)-x(1,2)];
%     J=abs(det(DF)); % calculation of J (if AREAS not precomputed)
    M_local=J*MATERIALS0(i)*[2 1 1; 1 2 1; 1 1 2]/24;
    A(ELEM(i,:),ELEM(i,:))=A(ELEM(i,:),ELEM(i,:))+A_local+M_local;
    % add local rhs
    b_local=F(i)*AREAS(i)/3;
    b(ELEM(i,:))=b(ELEM(i,:))+b_local;
end

Eliminate unknowns corresponding to Dirichlet nodes, modify the r. h. s. vector according to Neumann boundary conditions, and solve the resulting linear system:

In [ ]:
%% modifications due to boundary conditions
u=zeros(n_NODE,1);
u(DBOUNDARY)=DVALUE;
b=b-A*u;
for i=1:length(NVALUE)
    x=NODE(NBOUNDARY(i,:),:);
    b(NBOUNDARY(i,:))=b(NBOUNDARY(i,:))+1/2*norm(x(2,:)-x(1,:))*NVALUE(i);
end

Plot solution:

In [ ]:
%% solution of the resulting linear system and visualization
u(FREENODE)=A(FREENODE,FREENODE)\b(FREENODE);
figure; 
#trimesh(ELEM,coords1,coords2,u);
imagesc(reshape(u,nx,ny));
colorbar()

## Numerical experiments:
1) Consider reactive term, i.e. $k_0\neq0$.